In [1]:
%run '/mnt/d/Users/Admin/Projects/Machine_Learning/Yu_Xian/affine.py'
from osgeo import gdal

In [2]:
gcps = []
gcps_lst = []
for _ in range(4):
    px = np.random.uniform(0, 799)
    py = np.random.uniform(0, 599)

    X = np.random.uniform(-180, 180)
    Y = np.random.uniform(-90, 90)
    
    gcps_lst.append((X, Y, px, py))
    
    gcp = gdal.GCP(X, Y, 0, px, py)
    gcps.append(gcp)

In [3]:
for i, gcp in enumerate(gcps):
    print(f"GCP {i}:")
    print(f"  pixel,line = ({gcp.GCPPixel}, {gcp.GCPLine})")
    print(f"  X,Y,Z      = ({gcp.GCPX}, {gcp.GCPY}, {gcp.GCPZ})")

GCP 0:
  pixel,line = (286.3194698544767, 179.90326587910673)
  X,Y,Z      = (-36.305995794973285, 20.272646683617168, 0.0)
GCP 1:
  pixel,line = (433.5231564370652, 429.4386233816361)
  X,Y,Z      = (-107.5442642659645, 0.9027544300838741, 0.0)
GCP 2:
  pixel,line = (297.57930968062493, 19.038790946231934)
  X,Y,Z      = (-117.87832528760572, -26.574107491124813, 0.0)
GCP 3:
  pixel,line = (734.5881376368823, 24.92611128273711)
  X,Y,Z      = (-16.840484988951744, -56.03853248692091, 0.0)


In [4]:
gt = gdal.GCPsToGeoTransform(gcps)

In [5]:
gt

(-682.8097642108745,
 1.3423650396579028,
 0.6033314601864453,
 484.9732387582442,
 -0.8652876745696567,
 -0.5458651549213722)

In [20]:
res_lst = []
for (X, Y, px, py) in gcps_lst:
    pred_X, pred_Y = transform( (px, py), gt[0:3], gt[3:] )
    res_x = (pred_X - X)**2
    res_y = (pred_Y - Y)**2
    res = np.sqrt(res_x + res_y)
    print( (X, Y, px, py) )
    print(pred_X, pred_Y)
    res_lst.append(res)

(48.534894311398574, -6.050651590313535, 488.75314592540025, 122.09306877785725)
46.93796136658888 -4.585186221400278
(-116.84304937620061, 53.49505875780517, 231.26578572511642, 400.2647503148273)
-130.87434231585485 66.37122488046609
(-67.2011728509454, 25.735226510336716, 278.66280411717736, 454.27090418192824)
-34.6666301706378 -4.120908492908086
(54.37849404361734, -78.09606696793615, 298.1550013405363, 530.4697539907114)
37.47217724777687 -62.58156345626321


In [22]:
res_lst

[np.float64(2.1674370989033642),
 np.float64(19.043971108378408),
 np.float64(44.15750519162893),
 np.float64(22.94609698434637)]

In [4]:
gdal.Transformer()

Warning 6: 'GCP_REFINE' is an unexpected value for METHOD option of type string-select.
Warning 6: transformer options does not support option ORDER
Warning 6: transformer options does not support option TOLERANCE
Warning 6: transformer options does not support option MIN_GCPS


ValueError: not enough values to unpack (expected 4, got 2)

# Old Code

In [ ]:
# This is what CRS_compute_georef_equations does

def solve_lss_from_gcps(gcps_lst):

    # solves pixel to georef first
    # matrix A12 is from pixel to georef
    A12 = np.array([[1, px, py] for (X, Y, px, py) in gcps_lst])
    bX12 = np.array([X for (X, Y, px, py) in gcps_lst])
    bY12 = np.array([Y for (X, Y, px, py) in gcps_lst])

    E12, *_ = np.linalg.lstsq(A12, bX12, rcond=None)
    N12, *_ = np.linalg.lstsq(A12, bY12, rcond=None)

    # solves georef to pixel
    A21 = np.array([[1, X, Y] for (X, Y, px, py) in gcps_lst])
    bX21 = np.array([px for (X, Y, px, py) in gcps_lst])
    bY21 = np.array([py for (X, Y, px, py) in gcps_lst])

    E21, *_ = np.linalg.lstsq(A21, bX21, rcond=None)
    N21, *_ = np.linalg.lstsq(A21, bY21, rcond=None)
    
    return E12, N12, E21, N21

# This is what CRS_georef does

def transform(coords, E, N):
    x, y = coords[0], coords[1]

    X = E[0] + E[1] * x + E[2] * y
    Y = N[0] + N[1] * x + N[2] * y
    return X, Y

def py_worst_outlier(gcps_lst, E, N, dfTolerance):

    worst_index = -1
    worst_residual = -1.0

    for i, gcp in enumerate(gcps_lst):
        X, Y, px, py = gcp

        X_pred, Y_pred = transform((px, py), E, N)
        X_res = (X_pred - X)
        Y_res = (Y_pred - Y)
    
        Res = np.sqrt((X_res)**2 + (Y_res)**2)
        if (Res >= dfTolerance) and (Res > worst_residual):
            worst_residual = Res
            worst_index = i

    if worst_index == -1:
        return "No outliers"
    else:
        return worst_index

def py_rm_outliers(gcps_lst, dfTolerance, nGCPmin = 3):

    gcps_lst_cp = list(gcps_lst)
    E12, N12, E21, N21 = solve_lss_from_gcps(gcps_lst_cp)

    while len(gcps_lst_cp) > nGCPmin:
        worst_index = py_worst_outlier(gcps_lst_cp, E12, N12, dfTolerance)
        if worst_index == "No outliers":
            break
        else:
            del gcps_lst_cp[worst_index]
            E12, N12, E21, N21 = solve_lss_from_gcps(gcps_lst_cp)

    return gcps_lst_cp, E12, N12, E21, N21

def py_GCPTransformer(gcps_lst, bRefine, dfTolerance, nGCPmin = 3):

    if (bRefine == True):
        if nGCPmin < 4:
            return "Need 4 or more GCPs for bRefine to be True"
        else:
            gcps_lst_cp, E12, N12, E21, N21 = py_rm_outliers(gcps_lst, dfTolerance, nGCPmin)
            return gcps_lst_cp, E12, N12, E21, N21
    else:
        E12, N12, E21, N21 = solve_lss_from_gcps(gcps_lst)
        return gcps_lst, E12, N12, E21, N21